# 📘 Galileo + OpenTelemetry Tracing Guide (Python)

This notebook demonstrates how to send OpenTelemetry traces to Galileo using OpenInference.

🔗 See full documentation [here](https://v2galileo.mintlify.app/guides/logs/otel)

In [ ]:
pip install -U \
    langchain \
    openinference-instrumentation-langchain \
    opentelemetry-sdk \
    opentelemetry-exporter-otlp \
    opentelemetry-api \
    "opentelemetry-proto>=1.12.0"  -q

In [ ]:
GALILEO_API_KEY = "YOUR_API_KEY"
PROJECT_NAME = "YOUR-PROJECT-NAME"
LOG_STREAM_NAME = "my_log_stream"

## 🔐 Authentication and Header Setup

In [ ]:
import os

headers = {
    "Galileo-API-Key": GALILEO_API_KEY,
    "project": PROJECT_NAME,
    "logstream": LOG_STREAM_NAME,
}
# Format headers as a comma-separated key=value string
# Example: Galileo-API-Key=abc123,project_name=my_project,log_stream_name=my_run
os.environ['OTEL_EXPORTER_OTLP_TRACES_HEADERS'] = ",".join([f"{k}={v}" for k, v in headers.items()])

## ⚙️ OpenTelemetry Setup

In [ ]:
from opentelemetry.sdk import trace as trace_sdk
from opentelemetry import trace as trace_api
from opentelemetry.sdk.trace.export import BatchSpanProcessor, ConsoleSpanExporter
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from openinference.instrumentation.langchain import LangChainInstrumentor

# OTEL endpoint for Galileo
endpoint = "https://app.galileo.ai/api/galileo/otel/traces"

# Setup tracer provider
tracer_provider = trace_sdk.TracerProvider()
tracer_provider.add_span_processor(BatchSpanProcessor(OTLPSpanExporter(endpoint)))

# Optional: log locally
tracer_provider.add_span_processor(BatchSpanProcessor(ConsoleSpanExporter()))

# Register the provider
trace_api.set_tracer_provider(tracer_provider=tracer_provider)

# LangChain instrumentation
LangChainInstrumentor().instrument(tracer_provider=tracer_provider)

print("LangChain OTEL tracing initialized.")

OTEL tracing initialized.


# Demo

In [ ]:
import os
try:
  if not os.environ.get("OPENAI_API_KEY"):
    from google.colab import userdata
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
except:
  print("Please set the  OPENAI_API_KEY variable")

from langchain.prompts import PromptTemplate
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain_openai import OpenAI

llm = OpenAI()

prompt_1 = PromptTemplate.from_template(
    """Test Prompt 1:
    Question: '{question}'
    Answer: """
)

prompt_2 = PromptTemplate.from_template(
    """Test Prompt 2
Question: '{question}'
Scientific Explanation:
Answer: """
)

chain = (
    {"explanation": prompt_2 | llm | StrOutputParser(), "question": RunnablePassthrough()}
    | prompt_1
    | llm
    | StrOutputParser()
)

question = "Why is the sky blue? v1"
chain.invoke(question, config={"callbacks": []})